<a href="https://colab.research.google.com/github/kswalter1128/FIN654/blob/main/ChurchReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the packages
import pandas as pd
import os
from datetime import datetime
import numpy as np
os.chdir('/content/drive/MyDrive/Church Financial Report/Raw Data')

In [ ]:
files = os.listdir()
line = []
for f in files:
    with open(f, 'r') as d:
        data = d.readlines()
        for i in data:
            line.append(i)
dic = {}
dic['RawData'] = line
df = pd.DataFrame(dic)

In [ ]:
#Clean up the data
print("Starting Length",len(df['RawData']))
df = df[df['RawData'].str.contains('NATIVITY')==False]
df = df[df['RawData'].str.contains('Total for envelope')==False]
df = df[df['RawData'].str.contains('For Period:')==False]
df = df[df['RawData'].str.contains('Total Posted')==False]
df = df[df['RawData'].str.contains('Posted Contributions')==False]
df = df[df['RawData'].str.contains('Env. # Date')==False]
df = df[df['RawData'].str.startswith("\n")==False]
print("Ending Length",len(df['RawData']))

# Get the Envelope Number Column
df['Envelope Number']=df['RawData'].str.slice(0,6)
df['Envelope Number']=df['Envelope Number'].apply(lambda x: x.strip())
df['Envelope Number']=df['Envelope Number'].apply(lambda x: np.nan if x=="" else x)
df['Envelope Number']=df['Envelope Number'].fillna(method='ffill')
df.head()

#Convert Dates
df['Date']=df['RawData'].str.slice(7,17)
df['Date']=df['Date'].apply(lambda x: datetime.strptime(x,"%m/%d/%Y"))

#get the fund number
df['FundNmbr']=df['RawData'].str.slice(22,25)

#get the fund name
df['RawData']=df['RawData'].str.slice(25)
df['Fund Name']=df['RawData'].str.extract("(.+\s)(?:[\d,-]+[.][\d{2}])")
df['Fund Name']=df['Fund Name'].str.strip()

#get the amount
df['Amount']=df['RawData'].str.extract("([-]?(?!,$)[\d,]+[.]\d{2})")
df['Amount']=df['Amount'].apply(lambda x: x.replace(",",""))
df['Amount']=df['Amount'].astype(float)

df.head()

Starting Length 45060
Ending Length 31866


,RawData,Envelope Number,Date,FundNmbr,Fund Name,Amount
5,Memorial-June Helfrich 50.00 ...,8986,2022-02-20,313,Memorial-June Helfrich,50.0
7,General Fund-P 100.00 ...,24,2022-02-06,101,General Fund-P,100.0
8,General Benev.-P 40.00 ...,24,2022-02-06,201,General Benev.-P,40.0
11,General Fund-NP 50.00 ...,12,2022-02-06,102,General Fund-NP,50.0
12,General Fund-NP 25.00 ...,12,2022-02-20,102,General Fund-NP,25.0


In [ ]:
#remove RawData Column
df.drop(columns='RawData', inplace=True)
df.head()

,Envelope Number,Date,FundNmbr,Fund Name,Amount
5,8986,2022-02-20,313,Memorial-June Helfrich,50.0
7,24,2022-02-06,101,General Fund-P,100.0
8,24,2022-02-06,201,General Benev.-P,40.0
11,12,2022-02-06,102,General Fund-NP,50.0
12,12,2022-02-20,102,General Fund-NP,25.0
